# ROGEN Aging Study - Validation Analyses

This notebook contains statistical validation scripts for the ROGEN aging study, including:

1. **Frequency Validation** (A.2.1.7.1): Chi-square tests comparing allele frequencies in the ROGEN Longevity Cohort vs. general European population (gnomAD)
2. **Clock Precision Benchmark** (A.2.1.10.1): Mean Absolute Error (MAE) and R² validation of epigenetic clock predictions
3. **Permutation Test** (A.2.1.11.1): Empirical p-value calculation for phenotypic associations using permutation testing


In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency
from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from pathlib import Path

# Set style for better-looking plots
import seaborn as sns
sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300

# Set up output directory
output_dir = Path('../analysis')
output_dir.mkdir(exist_ok=True)

print(f"Output directory: {output_dir.absolute()}")


## 1. Frequency Validation (A.2.1.7.1)

**Concept**: Validate that the 70 Longevity SNPs identified in the ROGEN study show significantly different allele frequencies in the Longevity Cohort compared to the general European population (using gnomAD as a proxy).

**Metric**: Chi-Square Test / P-value

**Hypothesis**: Longevity-associated variants should have different allele frequencies in longevity-enriched cohorts compared to the general population.


In [ ]:
# Mock Data: 5 top SNPs as example
# "Ref_Allele_Count" vs "Alt_Allele_Count"
# Comparing ROGEN Longevity Cohort (Simulated) vs. gnomAD (General Euro)

data = {
    'SNP': ['rs429358 (APOE)', 'rs123 (FOXO3)', 'rs456 (SIRT1)', 'rs789 (AKT1)', 'rs101 (HSPA1A)'],
    'ROGEN_Alt_Count': [50, 120, 80, 45, 90],   # Frequencies in your specific cohort
    'ROGEN_Total': [1000, 1000, 1000, 1000, 1000],
    'gnomAD_Alt_Count': [150, 800, 600, 300, 700], # Frequencies in general population (usually much larger)
    'gnomAD_Total': [10000, 10000, 10000, 10000, 10000]
}

df = pd.DataFrame(data)

results = []

print(f"{'SNP':<20} | {'P-Value':<12} | {'Status'}")
print("-" * 45)

for index, row in df.iterrows():
    # Contingency table: [[Cohort_Alt, Cohort_Ref], [General_Alt, General_Ref]]
    obs = np.array([
        [row['ROGEN_Alt_Count'], row['ROGEN_Total'] - row['ROGEN_Alt_Count']],
        [row['gnomAD_Alt_Count'], row['gnomAD_Total'] - row['gnomAD_Alt_Count']]
    ])
    
    chi2, p, dof, ex = chi2_contingency(obs)
    
    status = "VALIDATED" if p < 0.05 else "Insignificant"
    results.append({'SNP': row['SNP'], 'P_Value': p, 'Status': status, 'Chi2': chi2})
    print(f"{row['SNP']:<20} | {p:.2e}     | {status}")

# Convert results to DataFrame for further analysis
results_df = pd.DataFrame(results)
print(f"\n✓ Validated SNPs: {sum(results_df['Status'] == 'VALIDATED')} / {len(results_df)}")


In [ ]:
# Visualization: P-value distribution
plt.figure(figsize=(10, 6))
colors = ['green' if p < 0.05 else 'gray' for p in results_df['P_Value']]
plt.barh(results_df['SNP'], -np.log10(results_df['P_Value']), color=colors)
plt.axvline(-np.log10(0.05), color='red', linestyle='--', linewidth=2, label='Significance Threshold (p=0.05)')
plt.xlabel('-log10(P-value)', fontsize=12)
plt.ylabel('SNP', fontsize=12)
plt.title('Frequency Validation: ROGEN Longevity Cohort vs. gnomAD\n(Chi-Square Test Results)', 
          fontsize=14, fontweight='bold')
plt.legend()
plt.tight_layout()
plt.savefig(output_dir / "Frequency_Validation.png", dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Plot saved to: {output_dir / 'Frequency_Validation.png'}")


## 2. Clock Precision Benchmark (A.2.1.10.1)

**Concept**: A correlation plot looks nice, but Mean Absolute Error (MAE) proves accuracy. This script calculates the error of the epigenetic clock pipeline.

**Metric**: MAE (Mean Absolute Error) and R² (Coefficient of Determination)

**Standard**: Clinical precision typically requires MAE < 4 years for epigenetic clocks.


In [ ]:
# Simulate validation data
# True Age vs. Predicted Age (by your pipeline)
true_age = np.array([25, 30, 45, 50, 65, 70, 80, 85, 90, 95])
predicted_age = np.array([27, 29, 44, 52, 63, 72, 79, 88, 91, 94])

# Calculate Metrics
mae = mean_absolute_error(true_age, predicted_age)
r2 = r2_score(true_age, predicted_age)

print("--- ROGEN Epigenetic Clock Validation Metrics ---")
print(f"Sample Size: {len(true_age)}")
print(f"Mean Absolute Error (MAE): {mae:.2f} years")
print(f"Correlation (R²): {r2:.4f}")

# Logic for the report
if mae < 4.0:
    print("\n✓ RESULT: Pipeline meets clinical precision standards (< 4 years error).")
else:
    print("\n⚠ RESULT: Pipeline requires calibration.")


In [ ]:
# Visualization: Predicted vs. True Age
plt.figure(figsize=(8, 8))
plt.scatter(true_age, predicted_age, s=100, alpha=0.7, color='steelblue', edgecolors='black', linewidth=1.5)

# Perfect prediction line (y=x)
min_age = min(min(true_age), min(predicted_age))
max_age = max(max(true_age), max(predicted_age))
plt.plot([min_age, max_age], [min_age, max_age], 'r--', linewidth=2, label='Perfect Prediction')

# Add MAE band (±MAE around perfect line)
plt.fill_between([min_age, max_age], 
                 [min_age - mae, max_age - mae], 
                 [min_age + mae, max_age + mae], 
                 alpha=0.2, color='gray', label=f'±MAE ({mae:.2f} years)')

plt.xlabel('True Age (years)', fontsize=12)
plt.ylabel('Predicted Age (years)', fontsize=12)
plt.title(f'ROGEN Epigenetic Clock Validation\nMAE = {mae:.2f} years, R² = {r2:.4f}', 
          fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(output_dir / "Clock_Precision_Validation.png", dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Plot saved to: {output_dir / 'Clock_Precision_Validation.png'}")


## 3. Permutation Test for Phenotypes (A.2.1.11.1)

**Concept**: You found a link between Age Acceleration and Inflammation. Was it luck? A Permutation Test shuffles the data 1000 times to prove your finding isn't random noise.

**Metric**: Empirical P-value

**Method**: Shuffle one variable randomly and recalculate correlation many times. The p-value is the proportion of permutations that yield correlations as extreme as the observed correlation.


In [ ]:
# Real correlation you found (e.g., between AgeAccel and IL-6)
real_correlation = 0.65

# Permutation Test
n_permutations = 1000
permuted_corrs = []

# Mock data vectors (in real analysis, these would come from your actual data)
np.random.seed(42)  # For reproducibility
vec_a = np.random.rand(50)
vec_b = vec_a * 0.5 + np.random.normal(0, 0.1, 50)  # correlated data

# Calculate the real correlation from the data
real_correlation = np.corrcoef(vec_a, vec_b)[0, 1]

print(f"Real Correlation: {real_correlation:.4f}")
print(f"Running {n_permutations} permutations...")

for i in range(n_permutations):
    # Shuffle one vector to break the biological link
    shuffled_b = np.random.permutation(vec_b)
    # Calculate correlation of noise
    corr = np.corrcoef(vec_a, shuffled_b)[0, 1]
    permuted_corrs.append(corr)

permuted_corrs = np.array(permuted_corrs)

# Calculate P-value (how many random shuffles were better than your real result?)
# Using two-tailed test: count permutations with |correlation| >= |real_correlation|
p_value = (np.sum(np.abs(permuted_corrs) >= np.abs(real_correlation)) + 1) / (n_permutations + 1)

print(f"\nPermutation Test Results:")
print(f"Real Correlation: {real_correlation:.4f}")
print(f"Permutation P-value: {p_value:.4f}")
print(f"Status: {'VALIDATED' if p_value < 0.05 else 'Not Significant'}")


In [ ]:
# Visualization of the permutation test
plt.figure(figsize=(10, 6))
plt.hist(permuted_corrs, bins=30, color='gray', alpha=0.5, label='Random Noise (Permutations)', edgecolor='black')
plt.axvline(real_correlation, color='red', linestyle='--', linewidth=2, label=f'ROGEN Finding (r={real_correlation:.4f})')
plt.axvline(-real_correlation, color='red', linestyle='--', linewidth=2, alpha=0.5)
plt.xlabel('Correlation Coefficient', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title(f'Permutation Test: Validation of Phenotypic Association\nP-value = {p_value:.4f}', 
          fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.savefig(output_dir / "Permutation_Validation.png", dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Plot saved to: {output_dir / 'Permutation_Validation.png'}")


## Summary

This notebook provides three key validation analyses for the ROGEN aging study:

1. **Frequency Validation**: Statistical comparison of allele frequencies between the longevity cohort and general population
2. **Clock Precision**: Accuracy metrics (MAE, R²) for the epigenetic clock pipeline
3. **Permutation Testing**: Empirical validation of phenotypic associations

All results and visualizations are saved to the `../analysis/` directory for inclusion in reports.
